In [47]:
!pip install pdf2image
!pip install pytesseract
!pip install pdfplumber

DEPRECATION: Loading egg at /opt/anaconda3/envs/py311/lib/python3.11/site-packages/python_magic-0.4.27-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/py311/lib/python3.11/site-packages/etllib-1.1-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/py311/lib/python3.11/site-packages/iso8601-2.1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /opt/anaconda3/envs/py311/lib/python3.11/site-packages/python_magic-0.4.

In [ ]:
import os
import re
import pandas as pd
import pdfplumber
import pytesseract
from PIL import Image
import pdf2image

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        
        if len(text.strip()) < 200:
            print(f"  -> Using OCR for {os.path.basename(pdf_path)}")
            images = pdf2image.convert_from_path(pdf_path, dpi=300)
            text = ""
            for image in images:
                text += pytesseract.image_to_string(image) + "\n"
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    return text

def clean_text(text):
    text = re.sub(r'\n\s*\n', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'(\d)\s*\n\s*(\d)', r'\1 \2', text)
    text = re.sub(r'([A-Z])\s*\n\s*([A-Z])', r'\1\2', text)
    return text

def dms_to_dd(dms_str):
    # Converts DMS string (48° 04' 27.510 N) to Decimal Degrees (48.074308)
    if not dms_str or dms_str == 'N/A':
        return 'N/A'
    
    try:
        parts = re.findall(r"(\d+(?:\.\d+)?)", dms_str)
        direction = re.search(r'([NSEW])', dms_str, re.I).group(1).upper()
        
        if len(parts) >= 3:
            degrees = float(parts[0])
            minutes = float(parts[1])
            seconds = float(parts[2])
            
            dd = degrees + (minutes / 60) + (seconds / 3600)
            
            if direction in ['S', 'W']:
                dd = -dd
                
            return round(dd, 6)
    except Exception:
        return dms_str
    
    return dms_str

def parse_well_info(text, filename):
    text = clean_text(text)
    
    data = {
        'source_file': filename,
        'well_file_no': 'N/A',
        'well_name': 'N/A',
        'api_number': 'N/A',
        'county': 'N/A',
        'section': 'N/A',
        'township': 'N/A',
        'range': 'N/A',
        'quarter_quarter': 'N/A',
        'footages': 'N/A',
        'latitude': 'N/A',
        'longitude': 'N/A',
        'operator': 'N/A',
        'spud_date': 'N/A',
        'completion_date': 'N/A',
        'stimulation_type': 'N/A',
        'field_name': 'N/A',
        'pool': 'N/A',
        'raw_text_sample': text[:500].replace('\n', ' ')
    }

    patterns = [
        r'Well File No\.?\s*[:\.\s]*(\d{4,6})\b',
        r'Well File\s*#?\s*(\d{4,6})\b',
        r'NDIC File Number\s*[:\.\s]*(\d{4,6})\b',
        r'State File No\.?\s*(\d{4,6})\b',
        r'File No\.?\s*[:\.\s]*(\d{4,6})\b',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            num = match.group(1)
            if num not in ['600', '900', '405', '58505', '80112']:
                data['well_file_no'] = num
                break

    patterns = [
        r'Well or Facility Name\s*[:\s]*([A-Z][A-Za-z\s&\-\#\d]+?)(?=\n|Field|Type|$)',
        r'Well Name and Number\s*\n?\s*([A-Z][A-Za-z\s&\-\#\d]+?\d+-\d+[A-Z]?)',
        r'Well Name\s*[:\s]*([A-Z][A-Za-z\s&\-\#\d]+?\d+-\d+[A-Z]?)',
        r'RE:\s*([A-Z][A-Za-z\s&\-\#\d]+?\d+-\d+[A-Z]?)',
        r'WELL:\s*([A-Z][A-Za-z\s&\-\#\d]+?\d+-\d+[A-Z]?)',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            name = match.group(1).strip()
            name = re.sub(r'\s+', ' ', name)
            if len(name) > 8 and not any(x in name for x in ['Qtr', 'Section', 'Township', 'Number', 'and Number']):
                data['well_name'] = name
                break

    patterns = [
        r'API\s*#?\s*[:\s]*(\d{2}-\d{3}-\d{5})',
        r'API[:\s]*(\d{2}-\d{3}-\d{5})',
        r'API\s*Number\s*[:\s]*(\d{2}-\d{3}-\d{5})',
        r'APIM?\s*[:\s]*(\d{2}-\d{3}-\d{5})',
        r'(\d{2}-\d{3}-\d{5})\s*\(',
        r'ST FILE NO\s*[:\s]*\d+\s*API\s*[:\s]*(\d{2}-\d{3}-\d{5})',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            data['api_number'] = match.group(1)
            break
    
    if data['api_number'] == 'N/A':
        match = re.search(r'33-053-\d{5}', text)
        if match:
            data['api_number'] = match.group(0)

    match = re.search(r'County\s*[:\s]*([A-Z][a-z]+)', text, re.IGNORECASE)
    if match:
        county = match.group(1).strip()
        if county.lower() not in ['section', 'township', 'range', 'lewis', 'basic', 'engineers']:
            data['county'] = county.title()
    if data['county'] == 'N/A' and re.search(r'McKenzie', text, re.IGNORECASE):
        data['county'] = 'McKenzie'

    match = re.search(r'Section\s*[:\s]*(\d{1,2})\b', text, re.IGNORECASE)
    if match:
        data['section'] = match.group(1)

    match = re.search(r'Township\s*[:\s]*(\d{3})\s*(N)\b', text, re.IGNORECASE)
    if match:
        data['township'] = match.group(1) + match.group(2)
    else:
        match = re.search(r'T(\d{3})N\s*-?\s*\d{3}W', text, re.IGNORECASE)
        if match:
            data['township'] = match.group(1) + 'N'
        else:
            match = re.search(r'(\d{3})\s*N\s*-?\s*\d{3}\s*W', text)
            if match:
                data['township'] = match.group(1) + 'N'
            else:
                match = re.search(r'Sec\s*\d+\s*,?\s*T\s*(\d{3})N', text, re.IGNORECASE)
                if match:
                    data['township'] = match.group(1) + 'N'

    match = re.search(r'Range\s*[:\s]*(\d{3})\s*(W)\b', text, re.IGNORECASE)
    if match:
        data['range'] = match.group(1) + match.group(2)
    else:
        match = re.search(r'\d{3}\s*N\s*-?\s*(\d{3})W', text, re.IGNORECASE)
        if match:
            data['range'] = match.group(1) + 'W'
        else:
            match = re.search(r'R(\d{3})W', text, re.IGNORECASE)
            if match:
                data['range'] = match.group(1) + 'W'
            else:
                match = re.search(r'T\d{3}N\s*-?\s*R(\d{3})W', text, re.IGNORECASE)
                if match:
                    data['range'] = match.group(1) + 'W'

    patterns = [
        r'Qtr-Qtr\s*[:\s]*([A-Z]{4})\b',
        r'Quarter-Quarter\s*[:\s]*([A-Z]{4})\b',
        r'(SENE|NENE|NWNE|SWNE|SENW|NENW|NWNW|SWNW|SESE|NESE|NWSE|SWSE|SWSW|NWSW)\s*\d',
        r'\b(SENE|NENE|NWNE|SWNE|SENW|NENW|NWNW|SWNW|SESE|NESE|NWSE|SWSE|SWSW|NWSW)\b',
        r'(NWNE|SENE|SWSW|NENE)\s*Sec',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            qq = match.group(1).upper()
            if len(qq) == 4 and qq in ['SENE', 'NENE', 'NWNE', 'SWNE', 'SENW', 'NENW', 'NWNW', 'SWNW', 'SESE', 'NESE', 'NWSE', 'SWSE', 'SWSW', 'NWSW']:
                data['quarter_quarter'] = qq
                break

    patterns = [
        r'SURF[:\s]*(\d+)\s*\'?\s*FNL\s*(\d+)\s*\'?\s*FEL',
        r'Surface Location\s*(\d+)\s*\'?\s*FNL\s*(\d+)\s*\'?\s*FEL',
        r'(\d+)\s*\'?\s*FNL\s*(\d+)\s*\'?\s*FEL',
        r'(\d+)\s*FNL\s*(\d+)\s*FEL',
        r'(\d+)\s*\'?\s*FSL\s*(\d+)\s*\'?\s*FWL',
        r'(\d+)\s*FSL\s*(\d+)\s*FWL',
        r'At Surface\s*(\d+)\s*FNL\s*(\d+)\s*FEL',
        r'At Surface\s*(\d+)\s*FSL\s*(\d+)\s*FWL',
        r'Surface Location\s*(\d+)\s*FSL\s*(\d+)\s*FWL',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            if 'FSL' in pattern or 'FSL' in text[match.start():match.end()]:
                data['footages'] = f"{match.group(1)}' FSL & {match.group(2)}' FWL"
            else:
                data['footages'] = f"{match.group(1)}' FNL & {match.group(2)}' FEL"
            break



    lat_match = re.search(r'Latitude\s*[:\s]*(\d{1,2}[^\d]+\d{1,2}[^\d]+\d+(?:\.\d+)?[^\w]*[NS])', text, re.I)
    if lat_match:
        raw_lat = re.sub(r'\s+', ' ', lat_match.group(1).strip())
        data['latitude'] = dms_to_dd(raw_lat)

    lon_match = re.search(r'Longitude\s*[:\s]*(\d{1,3}[^\d]+\d{1,2}[^\d]+\d+(?:\.\d+)?[^\w]*[EW])', text, re.I)
    if lon_match:
        raw_lon = re.sub(r'\s+', ' ', lon_match.group(1).strip())
        data['longitude'] = dms_to_dd(raw_lon)



    operators = []
    patterns = [
        r'Well Operator\s*[:\s]*([A-Z][A-Za-z\s&,\.]+?)(?=\n|Date|$)',
        r'Operator\s*[:\s]*([A-Z][A-Za-z\s&,\.]+?)(?=\n|Telephone|Address|Field|$)',
        r'PRODUCER\s*[:\s]*([A-Z][A-Za-z\s&,\.]+?)(?=\n|FIELD|$)',
        r'NAME OF OPERATOR\s*[:\s]*([A-Z][A-Za-z\s&,\.]+?)(?=\n|ADDRESS|$)',
        r'Company\s*[:\s]*([A-Z][A-Za-z\s&,\.]+?)(?=\n|Telephone|$)',
    ]
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        for m in matches:
            m = m.strip()
            if len(m) > 5 and not any(x in m for x in ['Representative', 'Telephone', 'Address']):
                operators.append(m)
    
    if operators:
        from collections import Counter
        operator_counts = Counter(operators)
        data['operator'] = operator_counts.most_common(1)[0][0]

    patterns = [
        r'Spud Date\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Spud Date\s*[:\s]*([A-Z][a-z]+\s+\d{1,2},?\s+\d{4})',
        r'Date Spudded\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'SPUDD[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Date Spud\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Spudded\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'SPUD DATE\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Date Spudded\s*[:\s]*([A-Z][a-z]+\s+\d{1,2},?\s+\d{4})',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            data['spud_date'] = match.group(1)
            break

    patterns = [
        r'Comp(?:letion)? Date\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Drilling Completed\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Date Comp\.\s*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'COMP DATE\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Date Comp\(ready to prod\)\s*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Comp Date\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
        r'Date Comp\s*[:\s]*(\d{1,2}/\d{1,2}/\d{2,4})',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            data['completion_date'] = match.group(1)
            break

    stim_keywords = []
    if re.search(r'\bAcidizing\b', text, re.IGNORECASE):
        stim_keywords.append('Acidizing')
    if re.search(r'\bFracture Treatment\b|\bFracturing\b|\bFrac\b', text, re.IGNORECASE):
        stim_keywords.append('Fracture Treatment')
    if re.search(r'\bShooting\b', text, re.IGNORECASE):
        stim_keywords.append('Shooting')
    if re.search(r'\bCement Squeeze\b', text, re.IGNORECASE):
        stim_keywords.append('Cement Squeeze')
    if stim_keywords:
        data['stimulation_type'] = ', '.join(stim_keywords)

    match = re.search(r'Field\s*(Name)?\s*[:\s]*([A-Z][A-Za-z]+)', text, re.IGNORECASE)
    if match:
        field = match.group(2).strip()
        if field.lower() not in ['county', 'section', 'township', 'range', 'rim', 'operating']:
            data['field_name'] = field
    if data['field_name'] == 'N/A' and re.search(r'\bBAKER\b', text, re.IGNORECASE):
        data['field_name'] = 'BAKER'

    match = re.search(r'Pool\s*[:\s]*([A-Z][A-Za-z]+)', text, re.IGNORECASE)
    if match:
        pool = match.group(1).strip()
        if pool.lower() not in ['madison', 'bakken', 'red river', 'duperow', 'mission canyon', 'ratcliff', 'nesson']:
            data['pool'] = pool.title()
    else:
        pools = ['Red River', 'Madison', 'Bakken', 'Duperow', 'Mission Canyon', 'Ratcliff', 'Nesson']
        for pool in pools:
            if re.search(rf'\b{pool}\b', text, re.IGNORECASE):
                data['pool'] = pool
                break

    return data

def main():
    pdf_folder = './DSCI560_Lab6_PDF'
    # pdf_folder = './Test_PDFs'
    output_csv = 'extracted_wells.csv'
    
    if not os.path.exists(pdf_folder):
        pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf') and f not in ['document.pdf']]
    else:
        pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

    extracted_data = []
    print(f"Found {len(pdf_files)} PDF files. Starting extraction...\n")

    for (i, pdf_path) in enumerate(pdf_files):
        print(f"Processing: {os.path.basename(pdf_path)}")
        text = extract_text_from_pdf(pdf_path)
        if text:
            well_info = parse_well_info(text, os.path.basename(pdf_path))
            extracted_data.append(well_info)
            print(f"  -> Well File No: {i}")
            print(f"  -> Well File No: {well_info['well_file_no']}")
            print(f"  -> Well Name: {well_info['well_name']}")
            print(f"  -> API: {well_info['api_number']}")
            print(f"  -> County: {well_info['county']}")
            print(f"  -> Section: {well_info['section']}, Twp: {well_info['township']}, Range: {well_info['range']}")
            print(f"  -> Qtr-Qtr: {well_info['quarter_quarter']}")
            print(f"  -> Footages: {well_info['footages']}")
            print(f"  -> Operator: {well_info['operator']}")
            print(f"  -> Spud: {well_info['spud_date']}, Comp: {well_info['completion_date']}")
            print(f"  -> Field: {well_info['field_name']}, Pool: {well_info['pool']}")
        else:
            print(f"  -> No text extracted")
        print()

    df = pd.DataFrame(extracted_data)
    df = df.replace('', 'N/A')
    df.to_csv(output_csv, index=False)
    
    print(f"\n{'='*70}")
    print(f"Extraction complete. Data saved to {output_csv}")
    print(f"{'='*70}\n")
    print(df[['source_file', 'well_file_no', 'well_name', 'api_number', 'county', 
              'section', 'township', 'range', 'quarter_quarter', 'footages',
              'operator', 'spud_date', 'completion_date', 'field_name', 'pool']].to_string())

if __name__ == "__main__":
    main()

Found 77 PDF files. Starting extraction...

Processing: W28654.pdf
  -> Well File No: 0
  -> Well File No: 28654
  -> Well Name: Spacing Unit Description Kline Federal 5300 41-18
  -> API: 33-053-06028
  -> County: Gas
  -> Section: 57, Twp: 153N, Range: 100W
  -> Qtr-Qtr: SWSW
  -> Footages: 200' FNL & 200' FEL
  -> Operator: Oasis Petroleum Oasis Petroleum
  -> Spud: N/A, Comp: N/A
  -> Field: Oasis, Pool: County

Processing: W22740.pdf
  -> Well File No: 1
  -> Well File No: 22099
  -> Well Name: Oasis Larry 5301 #44-12B
  -> API: 33-053-03911
  -> County: Gas
  -> Section: 57, Twp: 153N, Range: 101W
  -> Qtr-Qtr: SESE
  -> Footages: 197' FNL & 628' FEL
  -> Operator: M McCammond
  -> Spud: N/A, Comp: N/A
  -> Field: BAKERNORTH, Pool: Ed

Processing: W36047.pdf
  -> Well File No: 2
  -> Well File No: 36047
  -> Well Name: Lewis Federal 5300 11-31
  -> API: 33-053-08946
  -> County: Gas
  -> Section: 57, Twp: 153N, Range: 100W
  -> Qtr-Qtr: NWNW
  -> Footages: N/A
  -> Operator: Oasi